In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('ZOrderExample').getOrCreate()

In [23]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, shiftLeft, lit
from pyspark.sql.types import LongType


def compute_z_order(df, cols):
    df = df.withColumn("z_order", lit(0))

    for i, col_name in enumerate(cols):
        df = df.withColumn("z_order", shiftLeft(df.z_order, 1) | df[col_name].cast(LongType()))
    return df


In [21]:
# Here is an example DataFrame:
data = [("Alice", 1,2,3), ("Bob", 4,5,6), ("Charlie", 7,8,9)]
df = spark.createDataFrame(data, ["Name", "col1", "col2", "col3"])

# Using z-order function
df = compute_z_order(df, ["col1", "col2", "col3"])
df.show()

AnalysisException: [DATATYPE_MISMATCH.BINARY_OP_DIFF_TYPES] Cannot resolve "(shiftleft(z_order, 1) OR CAST(col1 AS BIGINT))" due to data type mismatch: the left and right operands of the binary operator have incompatible types ("INT" and "BIGINT").;
'Project [Name#47, col1#48L, col2#49L, col3#50L, (shiftleft(z_order#55, 1) OR cast(col1#48L as bigint)) AS z_order#61]
+- Project [Name#47, col1#48L, col2#49L, col3#50L, 0 AS z_order#55]
   +- LogicalRDD [Name#47, col1#48L, col2#49L, col3#50L], false
